In [250]:
import pandas as pd

In [251]:
papers = pd.read_csv('Papers.csv')
people = pd.read_csv('People.csv')

In [252]:
# people = people.loc[people['Birth Country'].isin(['America', 'England', 'France', 'Germany', 'Ireland', 'Scotland'])]

In [253]:
people_sampled = people.sample(49)
benjamin = people[people['Name']=='Benjamin Franklin']
people_sampled = pd.concat([people_sampled, benjamin])
people_sampled = people_sampled.drop(columns=['Kin?', 'Gender', 'BirthPlace', 'Smallest Birth Place', 'Birth State'])
people_sampled = people_sampled.dropna()
people_sampled.BirthYear = people_sampled.BirthYear.astype(int)
people_sampled.DeathYear = people_sampled.DeathYear.astype(int)
people_sampled.head()

,Name,BirthYear,DeathYear,Birth Country
243,Hannah Smith,1720,1761,America
219,George Washington,1732,1799,America
228,Giambattista Toderini,1728,1799,Italy
689,Thomas Ringgold,1715,1772,America
102,Charles-Guillaume-Frédéric Dumas,1721,1796,Germany


In [254]:
names = people_sampled['Name'].tolist()

In [255]:
paper_sampled = papers[papers['Author'].isin(names) & papers['Recipient'].isin(names)]
paper_sampled = paper_sampled[['DocumentID', 'DocumentTitle', 'Letter Year', 'Author', 'Recipient']]
paper_sampled.count()

DocumentID       83
DocumentTitle    83
Letter Year      83
Author           83
Recipient        83
dtype: int64

In [256]:
paper_sampled['type'] = paper_sampled.apply(lambda row: 'From' if row['Author']=='Benjamin Franklin' else 'To',axis=1)

In [257]:
paper_sampled

,DocumentID,DocumentTitle,Letter Year,Author,Recipient,type
49,623593,To Henry Bouquet,1757,Benjamin Franklin,Henry Bouquet,From
88,623632,From David Colden,1757,David Colden,Benjamin Franklin,To
191,623735,From David Colden,1758,David Colden,Benjamin Franklin,To
269,623813,To Sir Alexander Dick,1759,Benjamin Franklin,Alexander Dick,From
282,623826,To Sir Alexander Dick,1760,Benjamin Franklin,Alexander Dick,From
344,623888,To Sir Alexander Dick,1760,Benjamin Franklin,Alexander Dick,From
449,623993,To Thomas Ringgold,1761,Benjamin Franklin,Thomas Ringgold,From
473,624017,To Sir Alexander Dick,1762,Benjamin Franklin,Alexander Dick,From
534,624078,To Sir Alexander Dick,1762,Benjamin Franklin,Alexander Dick,From
634,624178,To William Greene,1763,Benjamin Franklin,William Greene,From


In [258]:
data = { 'nodes':[], 'links':[]}

In [259]:
people_sampled.apply(lambda row: data['nodes'].append({\
                                                       'id':row['Name'],\
                                                       'label':row['Name'],\
                                                       'start':row['BirthYear'],\
                                                       'end':row['DeathYear'],\
                                                       'type':row['Birth Country']\
                                                       }), axis=1);

In [260]:
paper_sampled.apply(lambda row: data['links'].append({\
                                                       'id':row['DocumentID'],\
                                                       'label':row['DocumentTitle'],\
                                                       'start':row['Letter Year'],\
                                                       'end':row['Letter Year'],\
                                                       'source':row['Author'],\
                                                       'target':row['Recipient'],\
                                                       'type':row['type']\
                                                       }), axis=1);

In [261]:
import json
with open('Franklin.json', 'w') as outfile:
    json.dump(data, outfile)